In [1]:
from importlib import import_module
from pathlib import Path
import sys
sys.path.insert(0, "../")
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# reload modules
import importlib

import data
importlib.reload(data)

from data import read_data, read_labels, normalize_img

2025-11-04 19:50:33.087283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-04 19:50:33.145853: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-04 19:50:33.469927: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/FPGA/Intel/openvino/openvino_2022.3.0/tools/compile_tool:/opt/FPGA/Intel/openvino/openvino_20

In [2]:
# Read MNIST database
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()

# Peprocessing (Normalization)
print('Raw data pixel value range:', train_data.min(), 'to', train_data.max())
train_data, train_labels = normalize_img(train_data, train_labels)
test_data, test_labels = normalize_img(test_data, test_labels)

print('Normalized datatye: ', type(train_data))
print('Normalized data pixel value range:', train_data.numpy().min(), 'to', train_data.numpy().max())

Raw data pixel value range: 0 to 255


2025-11-04 19:50:35.530537: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-11-04 19:50:35.534373: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-11-04 19:50:35.534448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-11-04 19:50:35.534898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuil

Normalized datatye:  <class 'tensorflow.python.framework.ops.EagerTensor'>
Normalized data pixel value range: 0.0 to 1.0


In [3]:
# Select model
model_type = 'fc'  # Cambia a 'cnn' para usar la red convolucional

In [4]:
# Define the path where the model is saved
OUTPUT_PATH = Path(f'./../../../saved_model/mnist_{model_type}') 

# Load the model
model = load_model(OUTPUT_PATH / 'model.h5')

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                50240     
                                                                 
 re_lu_4 (ReLU)              (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 re_lu_5 (ReLU)              (None, 32)                0         
                                                                 
 dense_8 (Dense)             (None, 10)                330       
                                                                 
 Softmax1 (Activation)       (None, 10)               

In [5]:
# Get weights

W0,b0 = model.layers[1].get_weights()   # Dense(64)
W1,b1 = model.layers[3].get_weights()   # Dense(32)
W2,b2 = model.layers[5].get_weights()   # Dense(10)


In [6]:

def save_dense(W, b, prefix, output_path):
    # Keras: W [in_dim, out_dim] → transponer a [out_dim, in_dim]
    W_rm = W.T.astype('float32').copy()
    b32  = b.astype('float32').copy()
    W_rm.tofile(f'{output_path}/{prefix}_W.bin')
    b32.tofile(f'{output_path}/{prefix}_b.bin')


In [ ]:
OUTPUT_PATH=Path(f'./../../opencl/weights/fc_fp32')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
save_dense(W0,b0,'fc0', OUTPUT_PATH)   # 128 x 784 en fichero
save_dense(W1,b1,'fc1', OUTPUT_PATH)   # 32  x 128
save_dense(W2,b2,'fc2', OUTPUT_PATH)   # 10  x 32
